In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [84]:
train = pd.read_csv('../../PythonProjects/ToxicCommentClassifier/data/train.csv')
test = pd.read_csv('../../PythonProjects/ToxicCommentClassifier/data/train.csv')

In [85]:
train


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [86]:
from tensorflow.keras.layers import TextVectorization

In [87]:
X = train['comment_text']
y = train[train.columns[2:]].values

In [88]:
MAX_EMBEDDING = 50000

In [89]:
vectorizer = TextVectorization(max_tokens=MAX_EMBEDDING, standardize='lower_and_strip_punctuation',
                               output_sequence_length=500, output_mode='int')


In [90]:
vectorizer.adapt(X.values)

In [91]:
vectorizer_model = Sequential()
vectorizer_model.add(vectorizer)
vectorizer_model.save('toxic_comment_vectorizer', save_format='tf')

INFO:tensorflow:Assets written to: toxic_comment_vectorizer\assets


In [92]:
vectorizer = tf.keras.models.load_model('toxic_comment_vectorizer').layers[0]

In [17]:
vectorizer('get the chair')[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([  96,    2, 5739,    0,    0], dtype=int64)>

In [94]:
vectorized_text = vectorizer(X.values)

In [11]:
vectorized_text

<tf.Tensor: shape=(159571, 1000), dtype=int64, numpy=
array([[  643,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2506, ...,     0,     0,     0],
       [  425,   440,    70, ...,     0,     0,     0],
       ...,
       [32141,  7329,   383, ...,     0,     0,     0],
       [    5,    12,   533, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [95]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y)).cache().shuffle(10000).batch(16).prefetch(8)


In [160]:
batch_X, batch_y = dataset.as_numpy_iterator().next()

In [96]:
train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [97]:
train_generator = train.as_numpy_iterator()

In [98]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [99]:
model = Sequential()
model.add(Embedding(MAX_EMBEDDING+1, 32))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

In [100]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [101]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 32)          1600032   
                                                                 
 dropout_4 (Dropout)         (None, None, 32)          0         
                                                                 
 bidirectional_4 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_17 (Dense)            (None, 128)               8320      
                                                                 
 dense_18 (Dense)            (None, 256)               33024     
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                      

In [102]:
history = model.fit(train, epochs=2, validation_data=val)

Epoch 1/2
6981/6981 [==============================] - 583s 83ms/step - loss: 0.0636 - val_loss: 0.0502
Epoch 2/2
6981/6981 [==============================] - 580s 83ms/step - loss: 0.0471 - val_loss: 0.0490


In [45]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [46]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 1s 838ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [48]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [18]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [19]:
predictions_on_train = model.predict(dataset)

9974/9974 [==============================] - 130s 13ms/step


In [20]:
pd.DataFrame(predictions_on_train).describe()

,0,1,2,3,4,5
count,159571.000000,1.595710e+05,1.595710e+05,1.595710e+05,1.595710e+05,1.595710e+05
mean,0.086550,9.450840e-03,5.344269e-02,2.011611e-03,4.800259e-02,1.132697e-02
std,0.241585,4.433041e-02,1.960592e-01,6.959079e-03,1.674862e-01,4.344819e-02
min,0.000003,1.682636e-17,7.785377e-09,4.867190e-13,6.165766e-10,2.532111e-09
25%,0.001222,3.139219e-10,2.793243e-05,1.037078e-07,1.036664e-05,1.068711e-05
50%,0.003397,4.967421e-09,1.087743e-04,7.840576e-07,5.242181e-05,4.204440e-05
75%,0.013649,2.326054e-07,7.233509e-04,1.316312e-05,4.835770e-04,2.870105e-04
max,1.000000,4.173617e-01,9.998251e-01,8.691684e-02,9.768088e-01,3.417069e-01


In [103]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [104]:
for batch in train.as_numpy_iterator():
    X_true, y_true = batch
    yhat = model.predict(X_true)
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 32ms/step


...

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step


In [105]:
print(f'Train Set Precision: {pre.result().numpy()}\n, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Train Set Precision: 0.8953999280929565
, Recall:0.6218206882476807, Accuracy:0.47700902819633484


In [106]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [107]:
for batch in test.as_numpy_iterator():
    X_true, y_true = batch
    yhat = model.predict(X_true)
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 89ms/step


1/1 [==============================] - 0s 32ms/step


In [108]:
print(f'Test Set Precision: {pre.result().numpy()}\n, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Test Set Precision: 0.8721227645874023
, Recall:0.5815804600715637, Accuracy:0.46740221977233887


In [110]:
model.save('toxicity_Embedding500000x32_Dropout_BiLSTM64_Dense128_Dropout_Dense256_Dense128_Dense6_betteremb.h5')


In [63]:
model = tf.keras.models.load_model('toxicity_Embedding_Dropout_BiLSTM64_Dense128_Dropout_Dense256_Dense128_Dense6.h5')

In [78]:
x = vectorizer(input_str)
[x.numpy()]
# np.expand_dims(vectorizer(input_str), 0)

[array([  40,  719,    9,   14,    9,   14,    2,  108,   50, 2805,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [75]:
input_str = "What nonsense is this. Is this the way one talks?!"

res = model.predict(np.expand_dims(vectorizer(input_str), 0))
print(res)

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [82]:
from lime import lime_text

input_str = "What nonsense is this. Is this the way one talks?!"

def explainer_fn(input_str):
    toxic_prediction = model.predict(vectorizer(input_str))
    return toxic_prediction

explainer = lime_text.LimeTextExplainer()

exp = explainer.explain_instance(input_str, explainer_fn)

157/157 [==============================] - 5s 27ms/step


In [83]:
exp.as_list()

[('nonsense', 0.0007008321833596098),
 ('way', -0.00029901245650231265),
 ('Is', 0.00027465332554876674),
 ('is', 0.00026933048166928895),
 ('the', -0.00020789201961333558),
 ('What', -9.35498567184107e-05),
 ('talks', -8.808926053345626e-05),
 ('one', -4.9393589355879955e-05),
 ('this', 2.4454001148307476e-05)]

In [141]:
import nltk

In [150]:
word_list = {k:v for k, v in exp.as_list()}
new_str = ""
highest_toxic = min(word_list.values())
least_toxic = max(word_list.values())
for word in nltk.word_tokenize(input_str, ):
    if not word_list.get(word):
        continue
    new_str += "({}){} ".format(word, (highest_toxic - word_list.get(word))/(highest_toxic - least_toxic)*100)
new_str

'(What)-0.0 (nonsense)100.0 (is)45.6224532099243 (this)41.04821143235788 (Is)46.15545254266244 (this)41.04821143235788 (the)37.31718886262614 (way)26.528769657389695 (one)37.28025299419937 (talks)26.12471771652909 '

In [132]:
{k:v for k, v in exp.as_list()}

{'nonsense': 0.0006222118557506182,
 'What': -0.0005003967894214821,
 'talks': -0.00020711844980891923,
 'way': -0.00020258252778983243,
 'one': -8.188544636656924e-05,
 'the': -8.147080111444086e-05,
 'this': -3.9586019193310126e-05,
 'Is': 1.7748311040752474e-05,
 'is': 1.1764814452724444e-05}